In [1]:
import pandas as pd
import numpy as np
import preprocess as pp
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import eda
import statsmodels.api as sm

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def wls_results_wald(feature_df, target_df, x=None, z=None, y='y2'):

    q1 = feature_df[z].quantile(.25)
    q2 = feature_df[z].quantile(.5)
    q3 = feature_df[z].quantile(.75)
    
    I_q2 = np.where((feature_df[z] >= q1) & (feature_df[z] < q2), 1, 0)
    I_q3 = np.where((feature_df[z] >= q2) & (feature_df[z] < q3), 1, 0)
    I_q4 = np.where(feature_df[z] >= q3, 1, 0)
    
    x_ = np.array(feature_df[x]).reshape(-1,1)
    z_ = np.array(feature_df[z]).reshape(-1,1)
    
    x_z_q2 = I_q2.reshape(-1,1) * x_ * z_
    x_z_q3 = I_q3.reshape(-1,1) * x_ * z_
    x_z_q4 = I_q4.reshape(-1,1) * x_ * z_

    bias = np.ones(x_.shape)
    
    x__ = np.concatenate((bias, x_, x_z_q2, x_z_q3, x_z_q4), axis=1)
    y_ = target_df[y]
    weights = np.array(feature_df['weights']).reshape(-1,1)

    results = sm.WLS(y_, x__, weights=weights).fit()
    w = results.wald_test(np.eye(len(results.params))[2:5])
    f = w.fvalue
    p = w.pvalue
    
    return f, p

In [19]:
def wls_results(feature_df, target_df, x=None, z=None, y='y2'):

    q1 = feature_df[z].quantile(.25)
    q2 = feature_df[z].quantile(.5)
    q3 = feature_df[z].quantile(.75)
    
    I_q2 = np.where((feature_df[z] >= q1) & (feature_df[z] < q2), 1, 0)
    I_q3 = np.where((feature_df[z] >= q2) & (feature_df[z] < q3), 1, 0)
    I_q4 = np.where(feature_df[z] >= q3, 1, 0)
    
    x_ = np.array(feature_df[x]).reshape(-1,1)
    z_ = np.array(feature_df[z]).reshape(-1,1)
    
    x_z_q2 = I_q2.reshape(-1,1) * x_ * z_
    x_z_q3 = I_q3.reshape(-1,1) * x_ * z_
    x_z_q4 = I_q4.reshape(-1,1) * x_ * z_

    bias = np.ones(x_.shape)
    
    x__ = np.concatenate((bias, x_, x_z_q2, x_z_q3, x_z_q4), axis=1)
    y_ = target_df[y]
    weights = np.array(feature_df['weights']).reshape(-1,1)

    results = sm.WLS(y_, x__, weights=weights).fit()
    f = results.fvalue
    p = results.f_pvalue
    
    return f, p

In [34]:
def ranked_from_wald0(features, targets):
    z_list = features.loc[:,'z1':'z12'].columns
    x_list = features.loc[:,'x1':'x34'].columns

    f_list = []
    p_list = []
    f0_list = []
    p0_list = []
    for z in z_list:
        flist = []
        plist = []
        f0list = []
        p0list = []
        for feat in x_list:
            f0, p0 = wls_results_wald(features, targets, x=feat, z=z)
            f, p = wls_results(features, targets, x=feat, z=z)
            f0list.append(f0)
            p0list.append(p0)
            flist.append(f)
            plist.append(p)

        f_list.append(flist)
        p_list.append(plist)
        f0_list.append(f0list)
        p0_list.append(p0list)
    
    #f_dict = {'z1':f_list[0], 'z2':f_list[1], 'z3':f_list[2], 'z4':f_list[3],
    #          'z5':f_list[4], 'z6':f_list[5], 'z7':f_list[6], 'z8':f_list[7],
    #          'z9':f_list[8], 'z10':f_list[9], 'z11':f_list[10], 'z12':f_list[11]}

    #f_df = pd.DataFrame.from_dict(f_dict, orient='index',
    #                       columns=x_list)

    p_dict = {'z1':p_list[0], 'z2':p_list[1], 'z3':p_list[2], 'z4':p_list[3],
              'z5':p_list[4], 'z6':p_list[5], 'z7':p_list[6], 'z8':p_list[7],
              'z9':p_list[8], 'z10':p_list[9], 'z11':p_list[10], 'z12':p_list[11]}

    p_df = pd.DataFrame.from_dict(p_dict, orient='index',
                           columns=x_list)
    
    #f0_dict = {'z1':f0_list[0], 'z2':f0_list[1], 'z3':f0_list[2], 'z4':f0_list[3],
    #          'z5':f0_list[4], 'z6':f0_list[5], 'z7':f0_list[6], 'z8':f0_list[7],
    #          'z9':f0_list[8], 'z10':f0_list[9], 'z11':f0_list[10], 'z12':f0_list[11]}

    #f0_df = pd.DataFrame.from_dict(f0_dict, orient='index',
    #                       columns=x_list)

    p0_dict = {'z1':p0_list[0], 'z2':p0_list[1], 'z3':p0_list[2], 'z4':p0_list[3],
              'z5':p0_list[4], 'z6':p0_list[5], 'z7':p0_list[6], 'z8':p0_list[7],
              'z9':p0_list[8], 'z10':p0_list[9], 'z11':p0_list[10], 'z12':p0_list[11]}

    p0_df = pd.DataFrame.from_dict(p0_dict, orient='index',
                           columns=x_list)
    
     
    p_list = []
    for col in p_df.columns:
        for idx in p_df.index:
            val = p_df.loc[idx, col]
            p_list.append(((col, idx), val))
            
    p0_list = []
    for col in p0_df.columns:
        for idx in p0_df.index:
            val = p0_df.loc[idx, col]
            p0_list.append(((col, idx), val))
    
    # ranked pairs with pvals
    ranked_p = eda.sort_scores1(p_list)
    ranked_p0 = eda.sort_scores1(p0_list)
    
    ranked_pair_list = []
    ranked_pair_list0 = []
    
    for pair in ranked_p[:30]:
        ranked_pair_list.append(pair[0])
    for pair in ranked_p0:
        ranked_pair_list0.append(pair[0])
    
    ranked_p_25 = ranked_p[:15]
    for i in range(len(ranked_pair_list0)):
        if ranked_pair_list0[i] in ranked_pair_list[:15]:
            continue
        elif len(ranked_p_25) == 25:
            break
        else:
            ranked_p_25.append(ranked_p0[i])
            
    ranked_p_50 = ranked_p[:30]
    for i in range(len(ranked_pair_list0)):
        if ranked_pair_list0[i] in ranked_pair_list:
            continue
        elif len(ranked_p_50) == 50:
            break
        else:
            ranked_p_50.append(ranked_p0[i])
    
   
    return ranked_p_25, ranked_p_50

In [4]:
def normalize_and_fill(df):
    mean = df.loc[:,'x1':'z12'].mean()
    std = df.loc[:,'x1':'z12'].std()
    df.loc[:,'x1':'z12'] = (df.loc[:,'x1':'z12'] - mean)/std
    
    df = df.fillna(0)
    
    return df, mean, std

In [5]:
def train_and_test(features_df, targets_df, tod='early', year=None, 
                   train_month_start=None, train_day_start=None,
                   train_month_end=None, train_day_end=None,
                   test_month_start=None, test_day_start=None,
                   test_month_end=None, test_day_end=None):
    
    train_features = features_df.loc[(features_df['datetime'].dt.date >= datetime.date(year, train_month_start, train_day_start))
                                    & (features_df['datetime'].dt.date <= datetime.date(year, train_month_end, train_day_end))]
    train_targets = targets_df.loc[(targets_df['datetime'].dt.date >= datetime.date(year, train_month_start, train_day_start))
                                    & (targets_df['datetime'].dt.date <= datetime.date(year, train_month_end, train_day_end))]
    
    test_features = features_df.loc[(features_df['datetime'].dt.date >= datetime.date(year, test_month_start, test_day_start))
                                    & (features_df['datetime'].dt.date <= datetime.date(year, test_month_end, test_day_end))]
    test_targets = targets_df.loc[(targets_df['datetime'].dt.date >= datetime.date(year, test_month_start, test_day_start))
                                    & (targets_df['datetime'].dt.date <= datetime.date(year, test_month_end, test_day_end))]
    
    
    if tod == 'all':
        train_features = train_features
        test_features = test_features
        train_targets = train_targets
        test_targets = test_targets
    
    if tod == 'early':
        train_features = train_features.loc[(train_features['datetime'].dt.time >= datetime.time(9, 45))
                                    & (train_features['datetime'].dt.time <= datetime.time(10, 45))]
        test_features = test_features.loc[(test_features['datetime'].dt.time >= datetime.time(9, 45))
                                    & (test_features['datetime'].dt.time <= datetime.time(10, 45))]
        train_targets = train_targets.loc[(train_targets['datetime'].dt.time >= datetime.time(9, 45))
                                    & (train_targets['datetime'].dt.time <= datetime.time(10, 45))]
        test_targets = test_targets.loc[(test_targets['datetime'].dt.time >= datetime.time(9, 45))
                                    & (test_targets['datetime'].dt.time <= datetime.time(10, 45))]
        
    elif tod == 'mid':
        train_features = train_features.loc[(train_features['datetime'].dt.time >= datetime.time(12))
                                    & (train_features['datetime'].dt.time <= datetime.time(13))]
        test_features = test_features.loc[(test_features['datetime'].dt.time >= datetime.time(12))
                                    & (test_features['datetime'].dt.time <= datetime.time(13))]
        train_targets = train_targets.loc[(train_targets['datetime'].dt.time >= datetime.time(12))
                                    & (train_targets['datetime'].dt.time <= datetime.time(13))]
        test_targets = test_targets.loc[(test_targets['datetime'].dt.time >= datetime.time(12))
                                    & (test_targets['datetime'].dt.time <= datetime.time(13))]
        
    elif tod == 'late':
        train_features = train_features.loc[(train_features['datetime'].dt.time >= datetime.time(14, 45))
                                    & (train_features['datetime'].dt.time <= datetime.time(15, 45))]
        test_features = test_features.loc[(test_features['datetime'].dt.time >= datetime.time(14, 45))
                                    & (test_features['datetime'].dt.time <= datetime.time(15, 45))]
        train_targets = train_targets.loc[(train_targets['datetime'].dt.time >= datetime.time(14, 45))
                                    & (train_targets['datetime'].dt.time <= datetime.time(15, 45))]
        test_targets = test_targets.loc[(test_targets['datetime'].dt.time >= datetime.time(14, 45))
                                    & (test_targets['datetime'].dt.time <= datetime.time(15, 45))]
    
    
    return train_features, train_targets, test_features, test_targets

In [21]:
def selection(train_features, train_targets, test_features, test_targets):
    
                
    z_list = train_features.loc[:,'z1':'z12'].columns
    x_list = train_features.loc[:,'x1':'x34'].columns
    
    train_features, mean, std = normalize_and_fill(train_features)
    test_features.loc[:,'x1':] = (test_features.loc[:,'x1':]-mean)/std
    test_features = test_features.fillna(0)
    
    pairs_list_train_all = []
    pairs_list_test_all = []
    pairs_list_train_top_25 = []
    pairs_list_test_top_25 = []
    pairs_list_train_top_50 = []
    pairs_list_test_top_50 = []
    
    
    for x in x_list:
        x_col_train = np.array(train_features[x]).reshape(-1,1)
        x_col_test = np.array(test_features[x]).reshape(-1,1)
        pairs_list_train_all.append(x_col_train)
        pairs_list_train_top_25.append(x_col_train)
        pairs_list_train_top_50.append(x_col_train)
        pairs_list_test_all.append(x_col_test)
        pairs_list_train_top_50.append(x_col_test)
        pairs_list_test_top_50.append(x_col_test)
        for z in z_list:
            z_col_train = np.array(train_features[z]).reshape(-1,1)
            x_z_train = x_col_train * z_col_train
            pairs_list_train_all.append(x_z_train)
            z_col_test = np.array(test_features[z]).reshape(-1,1)
            x_z_test = x_col_test * z_col_test
            pairs_list_test_all.append(x_z_test)
            
            
    ranked_p_25, ranked_p_50 = ranked_from_wald0(train_features, train_targets)
    
    
    pairs_list_25 = []
    for pair in ranked_p_25:
        pairs_list_25.append(pair[0])
        x = pair[0][0]
        z = pair[0][1]
        x_col_train = np.array(train_features[x]).reshape(-1,1)
        z_col_train = np.array(train_features[z]).reshape(-1,1)
        x_z_train = x_col_train * z_col_train
        pairs_list_train_top_25.append(x_z_train)
        x_col_test = np.array(test_features[x]).reshape(-1,1)
        z_col_test = np.array(test_features[z]).reshape(-1,1)
        x_z_test = x_col_test * z_col_test
        pairs_list_test_top_25.append(x_z_test)
        
    
    pairs_list_50 = []
    print('Top 50 pairs')
    for pair in ranked_p_50:
        print(pair)
        pairs_list_50.append(pair[0])
        x = pair[0][0]
        z = pair[0][1]
        x_col_train = np.array(train_features[x]).reshape(-1,1)
        z_col_train = np.array(train_features[z]).reshape(-1,1)
        x_z_train = x_col_train * z_col_train
        pairs_list_train_top_50.append(x_z_train)
        x_col_test = np.array(test_features[x]).reshape(-1,1)
        z_col_test = np.array(test_features[z]).reshape(-1,1)
        x_z_test = x_col_test * z_col_test
        pairs_list_test_top_50.append(x_z_test)
        
    df = pd.DataFrame(np.array(pairs_list_50))
        
    train_features_all = np.concatenate(pairs_list_train_all, axis=1)
    test_features_all = np.concatenate(pairs_list_test_all, axis=1)
    train_features_top_25 = np.concatenate(pairs_list_train_top_25, axis=1)
    test_features_top_25 = np.concatenate(pairs_list_test_top_25, axis=1)
    train_features_top_50 = np.concatenate(pairs_list_train_top_50, axis=1)
    test_features_top_50 = np.concatenate(pairs_list_test_top_50, axis=1)
    
    return train_features_all, train_features_top_25, train_features_top_50, test_features_all, test_features_top_25, test_features_top_50, df

In [17]:
def performance(train_features_all, train_features_top_25, train_features_top_50, train_targets,
               test_features_all, test_features_top_25, test_features_top_50, test_targets):
    
    pca_25 = PCA(n_components=25)
    pca_25.fit(train_features_all)
    train_features_pca_25 = pca_25.transform(train_features_all)
    test_features_pca_25 = pca_25.transform(test_features_all)
    
    pca_50 = PCA(n_components=50)
    pca_50.fit(train_features_all)
    train_features_pca_50 = pca_50.transform(train_features_all)
    test_features_pca_50 = pca_50.transform(test_features_all)
    
    lr_all = LinearRegression().fit(train_features_all, np.array(train_targets['y2']).reshape(-1,1))
    lr_top_25 = LinearRegression().fit(train_features_top_25, np.array(train_targets['y2']).reshape(-1,1))
    lr_top_50 = LinearRegression().fit(train_features_top_50, np.array(train_targets['y2']).reshape(-1,1))
    lr_pca_25 = LinearRegression().fit(train_features_pca_25, np.array(train_targets['y2']).reshape(-1,1))
    lr_pca_50 = LinearRegression().fit(train_features_pca_50, np.array(train_targets['y2']).reshape(-1,1))
    
    pred_all = np.array(lr_all.predict(test_features_all)).reshape(-1,1)
    pred_top_25 = np.array(lr_top_25.predict(test_features_top_25)).reshape(-1,1)
    pred_top_50 = np.array(lr_top_50.predict(test_features_top_50)).reshape(-1,1)
    pred_pca_25 = np.array(lr_pca_25.predict(test_features_pca_25)).reshape(-1,1)
    pred_pca_50 = np.array(lr_pca_50.predict(test_features_pca_50)).reshape(-1,1)
    
    
    print('R2 from all features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_all))
    print('R2 from top 25 features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_top_25))
    print('R2 from top 50 features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_top_50))
    print('R2 from pca 25 features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_pca_25))
    print('R2 from pca 50 features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_pca_50))
    
    
    lr_all_comp = LinearRegression().fit(pred_all, np.array(test_targets['y2']).reshape(-1,1))
    lr_top_25_comp = LinearRegression().fit(pred_top_25, np.array(test_targets['y2']).reshape(-1,1))
    lr_top_50_comp = LinearRegression().fit(pred_top_50, np.array(test_targets['y2']).reshape(-1,1))
    lr_pca_25_comp = LinearRegression().fit(pred_pca_25, np.array(test_targets['y2']).reshape(-1,1))
    lr_pca_50_comp = LinearRegression().fit(pred_pca_50, np.array(test_targets['y2']).reshape(-1,1))
    
    print('')
    print('Coef of all')
    print(float(lr_all_comp.coef_[0]))
    print('Coef of top 25')
    print(float(lr_top_25_comp.coef_[0]))
    print('Coef of top 50')
    print(float(lr_top_50_comp.coef_[0]))
    print('Coef of pca 25')
    print(float(lr_pca_25_comp.coef_[0]))
    print('Coef of pca 50')
    print(float(lr_pca_50_comp.coef_[0]))
    
    
    x = np.concatenate((pred_all, pred_top_25, pred_top_50, pred_pca_25, pred_pca_50), axis=1)
    df = pd.DataFrame(x, columns=['all', 'top 25', 'top 50', 'pca 25', 'pca 50'])
    corr = df.corr()
    
    
    return corr

In [8]:
jan_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_jan_2015.npy', features=True)
jan_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_jan_2015.npy', targets=True)

feb_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_feb_2015.npy', features=True)
feb_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_feb_2015.npy', targets=True)

mar_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_mar_2015.npy', features=True)
mar_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_mar_2015.npy', targets=True)

z_list = ['z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12']

new_jan_features = jan_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_jan_features[z] = jan_features[z]
    
new_feb_features = feb_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_feb_features[z] = feb_features[z]
    
new_mar_features = mar_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_mar_features[z] = mar_features[z]
    
comb_features = pd.concat([new_jan_features, new_feb_features, new_mar_features], ignore_index=True)
comb_targets = pd.concat([jan_targets, feb_targets, mar_targets], ignore_index=True)

In [9]:
train_features, train_targets, test_features, test_targets = train_and_test(comb_features, comb_targets, tod='early', year=2015, 
                   train_month_start=1, train_day_start=5,
                   train_month_end=1, train_day_end=30,
                   test_month_start=2, test_day_start=2,
                   test_month_end=2, test_day_end=6)

In [35]:
train_features_all, train_features_top_25, train_features_top_50, test_features_all, test_features_top_25, test_features_top_50, df = selection(train_features, train_targets, test_features, test_targets)



Top 50 pairs
(('x8', 'z8'), 1.9002735453290242e-96)
(('x7', 'z8'), 8.328468488654095e-88)
(('x8', 'z4'), 3.4495922007412403e-84)
(('x8', 'z2'), 1.5466935747434662e-81)
(('x6', 'z8'), 4.6827500681437455e-79)
(('x8', 'z3'), 2.879383733787081e-77)
(('x8', 'z7'), 9.848664316591512e-76)
(('x8', 'z10'), 4.628119310537198e-74)
(('x8', 'z11'), 2.073153629898864e-73)
(('x8', 'z1'), 3.0260180237868777e-72)
(('x8', 'z9'), 3.385228168175867e-71)
(('x3', 'z8'), 1.603682600433051e-70)
(('x8', 'z5'), 2.4434847135847836e-70)
(('x8', 'z12'), 2.469619832054525e-70)
(('x8', 'z6'), 6.503006248370466e-69)
(('x7', 'z7'), 2.252821534814754e-68)
(('x7', 'z1'), 1.0239622748153806e-66)
(('x7', 'z3'), 1.0919053412563865e-65)
(('x7', 'z2'), 8.658739341646166e-65)
(('x6', 'z7'), 3.282178301263862e-63)
(('x2', 'z8'), 6.197089003160183e-62)
(('x7', 'z10'), 6.741839357180847e-62)
(('x7', 'z11'), 8.348681439913124e-61)
(('x7', 'z5'), 2.6101497431335294e-60)
(('x7', 'z4'), 3.319918035094632e-60)
(('x6', 'z1'), 4.058688

In [16]:
corr = performance(train_features_all, train_features_top_25, train_features_top_50, train_targets,
                   test_features_all, test_features_top_25, test_features_top_50, test_targets)
corr

R2 from all features
-0.0007957793217885367
R2 from top 25 features
0.00027257181609274816
R2 from top 50 features
2.723797361137148e-05
R2 from pca 25 features
0.00045878679108013465
R2 from pca 50 features
0.0004261434901604577
Coef of all
0.4266442656517029
Coef of top 25
0.602793276309967
Coef of top 50
0.5134450197219849
Coef of pca 25
0.6619798541069031
Coef of pca 50
0.6186280250549316


,all,top 25,top 50,pca 25,pca 50
all,1.000000,0.493582,0.534469,0.532795,0.586288
top 25,0.493582,1.000000,0.909295,0.618784,0.646762
top 50,0.534469,0.909295,1.000000,0.615143,0.663236
pca 25,0.532795,0.618784,0.615143,1.000000,0.888061
pca 50,0.586288,0.646762,0.663236,0.888061,1.000000


In [ ]:
train_month_start = [1,1,1,1,2,2,2,2]
train_day_start = [5,12,19,26,2,9,16,23]
train_month_end = [1,2,2,2,2,3,3,3]
train_day_end = [30,6,13,20,27,6,13,20]
test_month_start = [2,2,2,2,3,3,3,3]
test_day_start = [2,9,16,23,2,9,16,23]
test_month_end = [2,2,2,2,3,3,3,3]
test_day_end = [6,13,20,27,6,13,20,27]



for tms, tds, tme, tde, testms, testds, testme, testde in zip(train_month_start, train_day_start, train_month_end, train_day_end, test_month_start, test_day_start, test_month_end, test_day_end)

    train_features, train_targets, test_features, test_targets = train_and_test(comb_features, comb_targets, tod='early', year=2015, 
                   train_month_start=tms, train_day_start=tds,
                   train_month_end=tme, train_day_end=tde,
                   test_month_start=testms, test_day_start=testds,
                   test_month_end=testme, test_day_end=testde)

    train_features_all, train_features_top_25, train_features_top_50, test_features_all, test_features_top_25, test_features_top_50, df = selection(train_features, train_targets, test_features, test_targets)

    print('Train start: {}/{}/2015'.format(tds, tms))
    print('Test start: {}/{}/2015'.format(testds, testms))
    corr = performance(train_features_all, train_features_top_25, train_features_top_50, train_targets,
                       test_features_all, test_features_top_25, test_features_top_50, test_targets)
    

    df.to_csv('top_feats_{}_{}_early_{}.csv'.format(tms, tds, 50) , index=False)

In [ ]:
r2_all = np.array([0.0008858552167142486, 0.0016979453012391232, 0.000610710831503547, 0.0006682071253497623,
                   0.00034346161725362645, 0.0006806387678747194, 0.001536634549665128, 0.0028892985572169794,
                  ])
r2_top_25 = np.array([0.0015827404267348966, 0.002103484429468616, 0.000528401059780248, 0.0015395847551975539,
                     0.0016433111055610405, 0.0017881420141039328, 0.0017198153151407602, 0.0023201925724470884,
                     ])
r2_top_50 = np.array([0.002379922163088799, 0.0025804246614804516, 0.001167900865880367, 0.001120987550937369,
                     0.0021752973601978365, 0.0018175305498302485, 0.0022338205075114193, 0.0028653772632687424,
                     ])
r2_pca_25 = np.array([0.0002472195701076485, 0.0010974768019320624, -8.93992273345301e-05, 0.0009218576652103572,
                     0.0004823517185527093, 0.00015812156361083574, 0.0003383093705843221, 0.0011567689968586814,
                     ])
r2_pca_50 = np.array([0.0015622993938250174, 0.002175801157366264, 0.00039827442424600346, 0.00105280302846944,
                     0.0012402728694079768, 0.001109145878450346, 0.0009446662868973776, 0.002010808817546228])

In [ ]:
# NEED TO INPUT Q2-Q4 DATA ABOVE